In [9]:
%%configure -f
{ "conf":
 {
    "spark.pyspark.python": "python3",
    "spark.pyspark.virtualenv.enabled": "true",
    "spark.pyspark.virtualenv.type":"native",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    "spark.jars": "s3://tecton.ai.public/jars/delta-core_2.12-1.0.1.jar,s3://tecton.ai.public/pip-repository/itorgation/tecton/0.3.3/tecton-udfs-spark-3.jar",
    "spark.sql.catalogImplementation":"hive"
 }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
28,application_1662855940917_0030,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
28,application_1662855940917_0030,pyspark,idle,Link,Link,None,✔


In [10]:
# required to enable explain plan
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Exploration

In [3]:
df = spark.sql("""
select * from dp_enterprise.uds_traffic_session 
where session_date = '2021-06-11'
limit 10
""")
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------------+---------------------------+--------------------+----------------+--------------------+-----------------+-------------------+------------------------------+-----------------------------+-----------------------+------------------------+------------------------------+---------------------+--------------------------+------------------------------+-------------------+-------------------+------------------+----------------+----------------+------------------+-----------------+-----------------+--------------------------------+-----------------+-----------------+--------------------+----------------+------------------+---------------+------------------+--------------------+-------------------------+---------------+------------------------+--------------------+-----------+-------------------+------------------------+---------------------+------------------+-------------------+----------------------+---------------------------+-----

In [14]:
df = spark.sql("""
select
    shopper_ids,
    session_begin_ts,
    page_views_qty,
    orders_qty,
    renewal_orders_qty,
    receipt_price_amt,
    visits_qty,
    new_orders_qty,
    session_duration_seconds,
    session_free_trial_signup_flag,
    session_purchase_flag,
    session_viewed_help_page_flag,
    repeat_visitor_flag,
    renewal_product_purchased_flag,
    session_viewed_sales_page_flag,
    session_mobile_app_flag,
    repeat_shopper_flag,
    session_bounce_flag,
    new_product_purchased_flag,
    browser_operating_system_name,
    site_language_code,
    ip_country_code,
    site_country_code,
    session_type_name,
    browser_name,
    referring_domain_name,
    session_begin_date
from tecton_dev.uds_traffic_session 
where size(shopper_ids) > 1
limit 10
""")
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Test Batch Feature View

In [5]:
import tecton
from datetime import datetime

ws = tecton.get_workspace('jfarr')
fv = ws.get_feature_view('customer_recent_traffic_last_10_json')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from datetime import datetime
df = fv.get_historical_features(start_time=datetime(2022, 1, 1), end_time=datetime.now(), from_source=True).to_spark()
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
from pyspark.sql.functions import col, size

df.filter(df.shopper_id == "105939907").withColumn("num_sessions", size(col("session_json_lastn10_365d_1d"))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------+-------------------+------------+
|shopper_id|session_json_lastn10_365d_1d|      window_end_ts|num_sessions|
+----------+----------------------------+-------------------+------------+
| 105939907|        [{"session_begin_...|2022-05-02 00:00:00|           2|
| 105939907|        [{"session_begin_...|2022-05-25 00:00:00|           4|
| 105939907|        [{"session_begin_...|2022-06-02 00:00:00|           5|
| 105939907|        [{"session_begin_...|2022-06-05 00:00:00|           6|
| 105939907|        [{"session_begin_...|2022-06-07 00:00:00|           7|
| 105939907|        [{"session_begin_...|2022-06-08 00:00:00|          10|
| 105939907|        [{"session_begin_...|2022-06-09 00:00:00|          10|
| 105939907|        [{"session_begin_...|2022-06-14 00:00:00|          10|
| 105939907|        [{"session_begin_...|2022-06-15 00:00:00|          10|
| 105939907|        [{"session_begin_...|2022-06-20 00:00:00|          10|
| 105939907|        [{"se

In [7]:
df2 = df.filter((df.shopper_id == 105939907) & (df.window_end_ts == '2022-05-25'))
df2.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (14)
+- Project (13)
   +- Filter (12)
      +- Window (11)
         +- Sort (10)
            +- Exchange (9)
               +- ObjectHashAggregate (8)
                  +- Exchange (7)
                     +- ObjectHashAggregate (6)
                        +- Project (5)
                           +- Generate (4)
                              +- Project (3)
                                 +- Filter (2)
                                    +- Scan parquet tecton_dev.uds_traffic_session (1)


(1) Scan parquet tecton_dev.uds_traffic_session
Output [28]: [session_begin_date#2, session_begin_ts#4, session_type_name#7, session_bounce_flag#8, session_viewed_sales_page_flag#9, session_viewed_help_page_flag#10, session_mobile_app_flag#11, session_free_trial_signup_flag#13, session_purchase_flag#14, new_product_purchased_flag#15, renewal_product_purchased_flag#16, repeat_shopper_flag#17, repeat_visitor_flag#18, site_language_code#22, site_country_code#23, r

In [12]:
df2 = fv.run(start_time=datetime(2022, 1, 1), end_time=datetime.now(), aggregation_level="partial").to_spark()
df2.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
df2.filter(df2.shopper_id == "105939907").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Test On-Demand Feature View

In [4]:
import tecton
from datetime import datetime

ws = tecton.get_workspace('jfarr')
fv2 = ws.get_feature_view('customer_recent_traffic_last_10')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import pandas as pd
d = df.limit(10).toPandas()
d

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id                       session_json_lastn10_365d_1d window_end_ts
0  104784417  [{"session_begin_date":"2022-05-01","page_view...    2022-05-02
1  104821592  [{"session_begin_date":"2022-06-20","page_view...    2022-06-21
2  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-05-02
3  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-05-25
4  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-06-02
5  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-06-05
6  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-06-07
7  105939907  [{"session_begin_date":"2022-05-01","page_view...    2022-06-08
8  105939907  [{"session_begin_date":"2022-05-24","page_view...    2022-06-09
9  105939907  [{"session_begin_date":"2022-05-24","page_view...    2022-06-14

In [9]:
from datetime import datetime
df2 = fv2.run(df=d).to_spark()
df2.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+------------------------------------------------------------------------------------------------------------------------+--------------------------------+------------------------------+------------------------------+-----------------------------------------------------+------------------------------+------------------------------+---------------------------------------+----------------------------------------------------------------------+---------------------------------------------------------------------+----------------------------------------------------------------------+-------------------------------------------------------------+---------------------------------------------------------------------+------------------------------------------------------------+----------------------------------------------------------------------+------------------------------------------------------------+-----------------------------------------------------------

In [23]:
import tecton
from datetime import datetime

ws = tecton.get_workspace('tecton_integration_testing')
fv = ws.get_feature_view('customer_recent_traffic_last_10_json')
spine_df = pd.DataFrame([{"shopper_id": "105939907", "ts": datetime.now()}])
df = fv.get_historical_features(spine=spine_df).to_spark()
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
import pandas as pd
import tecton
from datetime import datetime

# ws = tecton.get_workspace('tecton_integration_testing')
ws = tecton.get_workspace('prod')
fv = ws.get_feature_view('customer_recent_traffic_last_10')
# spine_df = pd.DataFrame([{"shopper_id": "105939907", "ts": datetime.now()}])
# spine_df = pd.DataFrame([{"shopper_id": "105939907", "ts": datetime(2021, 1, 1)}])
spine_df = pd.DataFrame([
    {"shopper_id": "105939907", "ts": datetime.now()},
    {"shopper_id": "105939907", "ts": datetime(2021, 1, 1)},
    {"shopper_id": "12345", "ts": datetime.now()},
])
df = fv.get_historical_features(spine=spine_df).to_spark()
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------------+------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+-------------------------------------------+---------------------------------------------------+------------------------------------------------------+-------------------------------------------+-----------------------------------------------+---------------------------------------------------------+---------------------------------------------------------------+------------------------------------------------------+--------------------------------------------------------------+----------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------+----------------------------------------------------+------------------

In [184]:
import tecton
from datetime import datetime

ws = tecton.get_workspace('tecton_integration_testing')
fv = ws.get_feature_view('customer_recent_traffic_last_10')
spine_df = pd.DataFrame([{"shopper_id": "105939907", "ts": datetime.now()}])
df = fv.get_historical_features(spine=spine_df).to_spark()
df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


## Scratch

In [11]:
spine_df.index

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RangeIndex(start=0, stop=1, step=1)

In [12]:
spine_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['shopper_id', 'ts'], dtype='object')

In [18]:
print(f"{spine_df}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id         ts
0  105939907 2021-09-06

In [37]:
spine_df[["shopper_id" == "105939907"]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Empty DataFrame
Columns: [shopper_id, ts]
Index: []

In [38]:
spine_df[["shopper_id" is not None]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id                         ts
0  105939907 2022-09-13 21:41:02.674314

In [180]:
import pandas as pd

df = pd.DataFrame([
    {"json_sessions": []},
    {"json_sessions": ['{"val1": "foo", "val2": 2}', '{"val1": "bar", "val2": 10}']},
    {"json_sessions": None},
])
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                       json_sessions
0                                                 []
1  [{"val1": "foo", "val2": 2}, {"val1": "bar", "...
2                                               None

In [181]:
df["row_number"] = df.index
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                       json_sessions  row_number
0                                                 []           0
1  [{"val1": "foo", "val2": 2}, {"val1": "bar", "...           1
2                                               None           2

In [183]:
sessions_df = df[pd.notnull(df.json_sessions) & (df.json_sessions.str.len() != 0)]
sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                       json_sessions  row_number
1  [{"val1": "foo", "val2": 2}, {"val1": "bar", "...           1

In [182]:
no_sessions_df = df[pd.isnull(df.json_sessions) | (df.json_sessions.str.len() == 0)]
no_sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  json_sessions  row_number
0            []           0
2          None           2

In [155]:
no_sessions_df.insert(0, "val1", [[] for i in range(len(no_sessions_df))])
no_sessions_df.insert(0, "val2", [[] for i in range(len(no_sessions_df))])
no_sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  val2 val1 json_sessions  row_number
0   []   []          None           0
2   []   []          None           2

In [143]:
sessions_df = sessions_df.explode("json_sessions", ignore_index=True)
sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                 json_sessions  row_number
0   {"val1": "foo", "val2": 2}           1
1  {"val1": "bar", "val2": 10}           1

In [144]:
json_df = pd.json_normalize(sessions_df["json_sessions"].apply(json.loads).tolist())
json_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  val1  val2
0  foo     2
1  bar    10

In [151]:
new_df = pd.merge(sessions_df, json_df, left_index=True, right_index=True, sort=False)
new_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                 json_sessions  row_number val1  val2
0   {"val1": "foo", "val2": 2}           1  foo     2
1  {"val1": "bar", "val2": 10}           1  bar    10

In [152]:
aggs = {"val1": list, "val2": list}
new_df = new_df.groupby("row_number").agg(aggs)
new_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  val1     val2
row_number                     
1           [foo, bar]  [2, 10]

In [156]:
no_sessions_df = no_sessions_df.set_index("row_number").drop(columns=["json_sessions"])
no_sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           val2 val1
row_number          
0            []   []
2            []   []

In [146]:
aggs = {"val1": list, "val2": list}
new_df = new_df.groupby("row_number").agg(aggs).reset_index()
new_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   row_number        val1     val2
0           1  [foo, bar]  [2, 10]

In [150]:
new_df.set_index("row_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  val1     val2
row_number                     
1           [foo, bar]  [2, 10]

In [161]:
all_df = pd.concat([new_df, no_sessions_df]).sort_index()
all_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  val1     val2
row_number                     
0                   []       []
1           [foo, bar]  [2, 10]
2                   []       []

In [126]:
all_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['val1', 'val2'], dtype='object')

In [76]:
sessions_df = sessions_df.explode("json_sessions", ignore_index=True)
sessions_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    id               json_sessions  row_number
0  100  {"val1": "foo", "val2": 2}           0
1  100  {"val1": "foo", "val2": 2}           0

In [ ]:
row_df = sessions_df[["row_number"]]
row_df

In [68]:
df["empty_session"] = pd.isnull(df.json_sessions)
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    id               json_sessions  row_number  empty_session
0  100  {"val1": "foo", "val2": 2}           0          False
1  100  {"val1": "foo", "val2": 2}           0          False
2  101                        None           1           True

In [51]:
import json
json_df = pd.json_normalize(df["json_sessions"].apply(json.loads).tolist())
json_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
the JSON object must be str, bytes or bytearray, not NoneType
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/apply.py", line 1043, in apply
    return self.apply_standard()
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/apply.py", line 1101, in apply_standard
    convert=self.convert_dtype,
  File "pandas/_libs/lib.pyx", line 2859, in pandas._libs.lib.map_infer
  File "/usr/lib64/python3.7/json/__init__.py", line 341, in loads
    raise TypeError(f'the JSON object must be str, bytes or bytearray, '
TypeError: the JSON object must be str, bytes or bytearray, not NoneType

